In [1]:
# pip install tensorflow==2.15.1 

In [2]:
import os
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, TimeDistributed, Bidirectional, Concatenate, GlobalAveragePooling1D, AdditiveAttention
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [7]:
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, TimeDistributed, Bidirectional, Concatenate, GlobalAveragePooling1D, AdditiveAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
from tensorflow.keras import regularizers



from tensorflow.keras.layers import BatchNormalization

class TextClassifierHyperModel:
    def __init__(self, input_shape, embeddings_GloVe, num_classes, parallel_blocks):
        self.input_shape = input_shape
        self.embeddings_GloVe = embeddings_GloVe
        self.num_classes = num_classes
        self.parallel_blocks = parallel_blocks

    def build(self):
        sequence_input = Input(shape=(self.input_shape,), dtype='int32')
        embedded_sequences = Embedding(input_dim=self.embeddings_GloVe.shape[0],
                                       output_dim=self.embeddings_GloVe.shape[1],
                                       weights=[self.embeddings_GloVe],
                                       trainable=False)(sequence_input)

        conv_blocks = []
        lstm_blocks = []

        # Adjusted hyperparameters
        conv_filter_units = 32
        time_units = 96
        lstm_units = 80
        dense_units = 130
        lstm_dropout_rate = 0.02  # Increased dropout rate
        lstm_r_dropout_rate = 0.015
        dropout_rate = 0.1
        learning_rate = 0.002

        for _ in range(self.parallel_blocks):
            conv = Conv1D(filters=conv_filter_units, kernel_size=1, padding='same', strides=1)(embedded_sequences)
            # conv_dense = TimeDistributed(Dense(time_units))(conv)
            conv_dense = BatchNormalization()(conv)  # Batch Normalization after dense layer
            conv_blocks.append(conv_dense)
            lstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True, dropout=lstm_dropout_rate, recurrent_dropout=lstm_r_dropout_rate))(conv_dense)
            lstm_blocks.append(lstm)

        combined = Concatenate()(conv_blocks + lstm_blocks)
        attention_layer = AdditiveAttention(use_scale=True)
        attention_output = attention_layer([combined, combined], return_attention_scores=False)
        context_vector = GlobalAveragePooling1D()(attention_output)

        dense = Dense(units=dense_units)(context_vector)
        dropout = Dropout(dropout_rate)(dense)
        outdata = Dense(self.num_classes, activation='sigmoid')(dropout)
        model = Model(inputs=sequence_input, outputs=outdata)
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model

input_length = 20
num_classes = 1
parallel_blocks = 2
EPOCHS=20
BATCH_SIZE = 32

# Initialize Weights & Biases
# wandb.init(project="LSTM_CNN_Attention")

# Instantiate the model
hypermodel = TextClassifierHyperModel(input_length, embeddings_GloVe, num_classes, parallel_blocks)
model = hypermodel.build()

# Model summary
model.summary()

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)


history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE,
                    callbacks=early_stopping_callback)

# wandb.finish()

NameError: name 'context_vector' is not defined

In [5]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

61/61 [==============================] - 2s 30ms/step - loss: 0.7102 - accuracy: 0.7361
Test Loss: 0.710237443447113
Test Accuracy: 0.7361327409744263


In [6]:
best_model.save(r'C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model')

NameError: name 'best_model' is not defined